In [2]:
import pandas as pd
import seaborn as sns
import numpy as np

from os.path import join

In [3]:
PROJ_DIR = "/Volumes/projects_herting/LABDOCS/Personnel/Katie/deltaABCD_clustering/"
DATA_DIR = "data/"
FIGS_DIR = "figures/"
OUTP_DIR = "output/"

In [6]:
df = pd.read_csv(join(PROJ_DIR, DATA_DIR, "data_qcd.csv"), index_col=0, header=0)

In [7]:
deltasmri_complete = pd.concat([df.filter(regex='smri.*change_score'), 
                                df.filter(regex='mrisdp.*change_score')], axis=1).dropna()
deltarsfmri_complete = df.filter(regex='rsfmri.*change_score').dropna(how='any')
deltarsi_complete = df.filter(regex='dmri_rsi.*change_score').dropna()
deltadti_complete = df.filter(regex='dmri_dti.*change_score').dropna()

In [8]:
var_df = pd.concat([deltasmri_complete, deltadti_complete, deltarsi_complete, deltarsfmri_complete], axis=1)

In [9]:
img_modalities = {'smri': deltasmri_complete,
                  'fmri': deltarsfmri_complete,
                  'rsi': deltarsi_complete, 
                  'dti': deltadti_complete}

In [10]:
concepts = {'morph': ['thick', 
                      'area', 
                      'vol',
                      'gmvol',
                      'dtivol'],
            'cell': ['t1wcnt', 
                     'rsirni', 
                     'rsirnd',
                     'rsirnigm', 
                     'rsirndgm',
                     'dtifa', 
                     'dtimd',
                     'dtild', 
                     'dtitd'],
            'func':['var',
                    'c',
                     'cor']}

In [11]:
stat_df = pd.concat([var_df.mean(), 
                     var_df.std(), 
                     var_df.min(), 
                     var_df.max(), 
                     var_df.quantile(0.25), 
                     var_df.quantile(0.75)], 
                    axis=1)

In [12]:
stat_df.columns = ['mean', 'sdev', 'max', 'min', '25%', '75%']

In [13]:
for var in stat_df.index:
    #print(var)
    if 'mrisdp' in var:
        var_num = int(var.split('.')[0].split('_')[-1])
        stat_df.at[var, 'modality'] = 'smri'
        stat_df.at[var, 'atlas'] = 'dtx'
        if var_num <= 148:
            stat_df.at[var, 'measure'] = 'thick'
        elif var_num <= 450 and var_num >= 303:
            stat_df.at[var, 'measure'] = 'area'
        elif var_num < 604 and var_num >= 450:
            stat_df.at[var, 'measure'] = 'vol'
        elif var_num <= 1054 and var_num >= 907:
            stat_df.at[var, 'measure'] = 't1wcnt'
        elif var_num == 604:
            stat_df.at[var, 'measure'] = 'gmvol'
    elif '_' in var:
        var_list = var.split('.')[0].split('_')
        stat_df.at[var, 'modality'] = var_list[0]
        stat_df.at[var, 'measure'] = var_list[1]
        stat_df.at[var, 'atlas'] = var_list[2]
        stat_df.at[var, 'region'] = '_'.join(var_list[3:])

stat_df = stat_df[stat_df['measure'] != 't1w']
stat_df = stat_df[stat_df['measure'] != 't2w']

In [14]:
for i in stat_df.index:
    measure = stat_df.loc[i]['measure']
    measure = str(measure)
    if measure in concepts['morph']:
        stat_df.at[i,'concept'] = 'macrostructure'
    elif measure in concepts['cell']:
        stat_df.at[i,'concept'] = 'microstructure'
    elif measure in concepts['func']:
        stat_df.at[i,'concept'] = 'function'

In [15]:
stat_df[stat_df['measure'] == 'area'].sort_values('mean')

,mean,sdev,max,min,25%,75%,modality,measure,atlas,region,concept
smri_area_cdk_suplrh.change_score,-0.545485,2.844302,-24.450421,25.704796,-1.183452,0.104452,smri,area,cdk,suplrh,macrostructure
smri_area_cdk_supllh.change_score,-0.540392,2.455613,-26.416364,26.114805,-1.135065,0.060413,smri,area,cdk,supllh,macrostructure
smri_area_cdk_pclh.change_score,-0.497624,1.619170,-22.478504,28.207707,-1.087101,0.083660,smri,area,cdk,pclh,macrostructure
smri_area_cdk_ifpllh.change_score,-0.497584,1.959056,-19.592383,16.966355,-1.142573,0.295476,smri,area,cdk,ifpllh,macrostructure
smri_area_cdk_pcrh.change_score,-0.486155,1.772318,-23.816851,30.311408,-1.076555,0.074083,smri,area,cdk,pcrh,macrostructure
...,...,...,...,...,...,...,...,...,...,...,...
smri_area_cdk_rracaterh.change_score,0.579390,3.541315,-22.444679,33.229329,-1.309258,2.471989,smri,area,cdk,rracaterh,macrostructure
smri_area_cdk_parahpallh.change_score,0.589456,2.842211,-21.279461,51.535938,-1.027668,2.180917,smri,area,cdk,parahpallh,macrostructure
smri_area_cdk_parahpalrh.change_score,0.680540,2.764852,-22.127174,19.744681,-0.955414,2.346570,smri,area,cdk,parahpalrh,macrostructure
smri_area_cdk_cdacaterh.change_score,0.700397,2.678528,-20.597407,38.581911,-0.776141,2.186506,smri,area,cdk,cdacaterh,macrostructure


In [65]:
def assign_region_names(df, region_col, missing=False):
    '''
    Input: 
    df = dataframe (variable x columns) with column containing region names in ABCD var ontology, 
    region_col = name of column containing region names.
    Output: 
    df = same dataframe, but with column mapping region variables to actual region names
    missing = optional, list of ABCD region names not present in region_names dictionary
    '''
    region_names = {'aal': ('Accumbens', 'L'), 
                    'aalh': ('Accumbens', 'L'), 
                    'aar': ('Accumbens', 'R'), 
                    'aarh': ('Accumbens', 'R'), 
                    'ablh': ('Accumbens', 'L'), 
                    'abrh': ('Accumbens', 'R'),
                    'ad': ('Auditory Network', 'B'), 
                    'aglh': ('Amygdala', 'L'), 
                    'agrh': ('Amygdala', 'R'), 
                    'amygdalalh': ('Amugdala', 'L'), 
                    'amygdalarh': ('Amygdala', 'R'), 
                    'aomtmlh': (),
                    'aomtmrh': (), 
                    'atrlh': ('Anterior Thalamic Radiation', 'L'), 
                    'atrrh': ('Anterior Thalamic Radiation', 'R'),
                    'bstslh': ('Banks of Superior Temporal Sulcus', 'L'),
                    'bstsrh': ('Banks of Superior Temporal Sulcus', 'R'),
                    'banksstslh': ('Banks of Superior Temporal Sulcus', 'L'), 
                    'banksstsrh': ('Banks of Superior Temporal Sulcus', 'R'),
                    'brainstem': ('Brainstem', 'B'), 
                    'bs': ('Brainstem', 'B'), 
                    'bstem': ('Brainstem', 'B'), 
                    'ca': ('Cinguloparietal Network', 'B'), 
                    'caclh': ('Cingulate Gyrus, Caudal Anterior', 'L'),
                    'cacrh': ('Cingulate Gyrus, Caudal Anterior', 'L'),
                    'caudatelh': ('Caudate', 'L'), 
                    'caudaterh': ('Caudate', 'R'),
                    'cbclh': ('Cerebellar Cortex', 'L'), 
                    'cbcrh': ('Cerebellar Cortex', 'R'), 
                    'cbwmlh': ('Cerebellar White Matter', 'L'), 
                    'cbwmrh': ('Cerebellar White Matter', 'R'), 
                    'cc': ('Corpus Callosum', 'B'), 
                    'cdacatelh': ('Anterior Cingulate, Caudal', 'L'),
                    'cdacaterh': ('Anterior Cingulate, Caudal', 'R'), 
                    'cdaclatelh': ('Anterior Cingulate, Caudal', 'L'), 
                    'cdaclaterh': ('Anterior Cingulate, Caudal', 'R'), 
                    'cdelh': ('Caudate', 'L'), 
                    'cderh': ('Caudate', 'R'), 
                    'cdlh': ('Caudate', 'L'),
                    'cdmdflh': ('Middle Frontal Gyrus, Caudal', 'L'), 
                    'cdmdfrh': ('Middle Frontal Gyrus, Caudal', 'R'), 
                    'cdmdfrlh': ('Middle Frontal Gyrus, Caudal', 'L'), 
                    'cdmdfrrh': ('Middle Frontal Gyrus, Caudal', 'R'), 
                    'cdrh': ('Caudate', 'R'), 
                    'cgc': ('Cingulo-Opercular Network', 'B'),
                    'cgclh': ('Cingulate Cingulum', 'L'), 
                    'cgcrh': ('Cingulate Cingulum', 'R'), 
                    'cghlh': ('Parahippocampal Cingulum', 'L'), 
                    'cghrh': ('Parahippocampal Cingulum', 'R'),
                    'cmflh': ('Middle Frontal Gyrus, Caudal', 'L'),
                    'cmfrm': ('Middle Frontal Gyrus, Caudal', 'R'),
                    'crbcortexlh': ('Cerebellar Cortex', 'L'),
                    'crbcortexrh': ('Cerebellar Cortex', 'R'), 
                    'crbwmatterlh': ('Cerebellar White Matter', 'L'), 
                    'crbwmatterrh': ('Cerebellar White Matter', 'L'), 
                    'crcxlh': ('Cerebellar Cortex', 'L'), 
                    'crcxrh': ('Cerebellar Cortex', 'R'),
                    'cstlh': ('Corticospinal Tract', 'L'), 
                    'cstrh': ('Corticospinal Tract', 'R'), 
                    'cuneuslh': ('Cuneus', 'L'), 
                    'cuneusrh': ('Cuneus', 'R'), 
                    'cwmlh': ('Cerebral White Matter', 'L'), 
                    'cwmrh': ('Cerebral White Matter', 'L'), 
                    'dla': ('Dorsal Attention Network', 'B'),
                    'dlprefrlh': ('Dorsal Prefrontal Cortex', 'L'), 
                    'dlprefrrh': ('Dorsal Prefrontal Cortex', 'R'), 
                    'dmfrlh': ('Dorsomedial Frontal Cortex', 'L'), 
                    'dmfrrh': ('Dorsomedial Frontal Cortex', 'R'), 
                    'dt': ('Default Mode Network', 'B'), 
                    'df': ('Default Mode Network', 'B'), 
                    'ehinallh': ('Entorhinal Cortex', 'L'),
                    'ehinalrh': ('Entorhinal Cortex', 'R'), 
                    'entorhinallh': ('Entorhinal Cortex', 'L'), 
                    'entorhinalrh': ('Entorhinal Cortex', 'R'), 
                    'fflh': ('Fusiform Gyrus', 'L'), 
                    'ffrh': ('Fusiform Gyrus', 'R'), 
                    'fmaj': ('Fornix Major', 'B'),
                    'fmin': ('Fornix Minor', 'B'), 
                    'fo': ('Frontoparietal Network', 'B'), 
                    'fpolelh': ('Frontal Pole', 'L'), 
                    'fpolerh': ('Frontal Pole', 'R'), 
                    'frpolelh': ('Frontal Pole', 'L'), 
                    'frpolerh': ('Frontal Pole', 'R'),
                    'fscslh': ('Superior Corticostriate Tract (Frontal)', 'L'), 
                    'fscsrh': ('Superior Corticostriate Tract (Frontal)', 'R'), 
                    'fusiformlh': ('Fusiform Gyrus', 'L'), 
                    'fusiformrh': ('Fusiform Gyrus', 'R'), 
                    'fxcutlh': ('Fornix (excluding Fimbria)', 'L'),
                    'fxcutrh': ('Fornix (excluding Fimbria)', 'R'), 
                    'fxlh': ('Fornix', 'L'), 
                    'fxrh': ('Fornix', 'R'), 
                    'hclh': ('Hippocampus', 'L'),
                    'hcrh': ('Hippocampus', 'R'), 
                    'hplh': ('Hippocampus', 'L'), 
                    'hprh': ('Hippocampus', 'R'), 
                    'hpuslh': ('Hippocampus', 'L'), 
                    'hpusrh': ('Hippocampus', 'R'), 
                    'ifolh': ('Inferior Fronto-occipital Fasciculus', 'L'), 
                    'iforh': ('Inferior Fronto-occipital Fasciculus', 'R'),
                    'ifpalh': ('Inferior Parietal', 'L'), 
                    'ifparh': ('Inferior Parietal', 'R'), 
                    'ifpllh': ('Inferior Parietal', 'L'), 
                    'ifplrh': ('Inferior Parietal', 'R'), 
                    'ifsfclh': ('Inferior Frontal Superior Frontal', 'L'), 
                    'ifsfcrh': ('Inferior Frontal Superior Frontal', 'R'),
                    'iftlh': ('Inferior Temporal', 'L'), 
                    'iftmlh': ('Inferior Temporal', 'L'), 
                    'iftmrh': ('Inferior Temporal', 'R'), 
                    'iftrh': ('Inferior Temporal', 'R'), 
                    'ihcatelh': ('Cingulate Gyrus, Ithsmus', 'L'), 
                    'ihcaterh': ('Cingulate Gyrus, Ithsmus', 'R'),
                    'ihclatelh': ('Cingulate Gyrus, Ithsmus', 'L'), 
                    'ihclaterh': ('Cingulate Gyrus, Ithsmus', 'R'), 
                    'ilflh': ('Inferior Longitudinal Fasiculus', 'L'), 
                    'ilfrh': ('Inferior Longitudinal Fasiculus', 'R'), 
                    'ilvlh': ('Inferior Lateral Ventricle', 'L'), 
                    'ilvrh': ('Inferior Lateral Ventricle', 'R'),
                    'insulalh': ('Insula', 'L'), 
                    'insularh': ('Insula', 'R'), 
                    'intracranialv': ('Intracranial Volume', 'B'), 
                    'linguallh': ('Lingual Gyrus', 'L'), 
                    'lingualrh': ('Lingual Gyrus', 'R'),
                    'lobfrlh': ('Orbitofrontal Gyrus, Lateral', 'L'), 
                    'lobfrrh': ('Orbitofrontal Gyrus, Lateral', 'R'), 
                    'loboflh': ('Orbitofrontal Gyrus, Lateral', 'L'), 
                    'lobofrh': ('Orbitofrontal Gyrus, Lateral', 'R'), 
                    'loccipitallh': ('Occipital Gyrus, Lateral', 'L'),
                    'loccipitalrh': ('Occipital Gyrus, Lateral', 'R'), 
                    'locclh': ('Occipital Gyrus, Lateral', 'L'), 
                    'loccrh': ('Occipital Gyrus, Lateral', 'R'), 
                    'lvrh': ('Lateral Ventricle', 'R'), 
                    'mdtlh': ('Middle Temporal Gyrus', 'L'), 
                    'mdtmlh': ('Middle Temporal Gyrus', 'L'),
                    'mdtmrh': ('Middle Temporal Gyrus', 'R'), 
                    'mdtrh': ('Middle Temporal Gyrus', 'R'), 
                    'mobfrlh': ('Occipital Gyrus, Medial', 'L'), 
                    'mobfrrh': ('Occipital Gyrus, Medial', 'R'), 
                    'moboflh': ('Occipital Gyrus, Medial', 'L'), 
                    'mobofrh': ('Occipital Gyrus, Medial', 'R'), 
                    'n': ('Extra-Network', 'B'),
                    'pallidumlh': ('Pallidum', 'L'), 
                    'pallidumrh': ('Pallidum', 'R'),
                    'paracentrallh': ('Paracentral Gyrus', 'L'), 
                    'paracentralrh': ('Paracentral Gyrus', 'R'), 
                    'paracnlh': ('Paracentral Gyrus', 'L'), 
                    'paracnrh': ('Paracentral Gyrus', 'R'),
                    'parahpallh': ('Parahippocampal Gyrus', 'L'), 
                    'parahpalrh': ('Parahippocampal Gyrus', 'R'), 
                    'parsobalislh': ('Inferior Frontal Gyrus, Pars Orbitalis', 'L'), 
                    'parsobalisrh': ('Inferior Frontal Gyrus, Pars Orbitalis', 'R'),
                    'parsobislh': ('Inferior Frontal Gyrus, Pars Orbitalis', 'L'), 
                    'parsobisrh': ('Inferior Frontal Gyrus, Pars Orbitalis', 'R'), 
                    'parsopclh': ('Inferior Frontal Gyrus, Pars Opercularis', 'L'), 
                    'parsopcrh': ('Inferior Frontal Gyrus, Pars Opercularis', 'R'), 
                    'parsopllh': ('Inferior Frontal Gyrus, Pars Opercularis', 'L'),
                    'parsoplrh': ('Inferior Frontal Gyrus, Pars Opercularis', 'R'), 
                    'parstgrislh': ('Inferior Frontal Gyrus, Pars Triangularis', 'L'), 
                    'parstgrisrh': ('Inferior Frontal Gyrus, Pars Triangularis', 'R'), 
                    'parstularislh': ('Inferior Frontal Gyrus, Pars Triangularis', 'L'),
                    'parstularisrh': ('Inferior Frontal Gyrus, Pars Triangularis', 'R'), 
                    'pclh': ('Precuneus', 'L'), 
                    'pcrh': ('Precuneus', 'R'), 
                    'pericclh': ('Pericalcarine Cortex', 'L'), 
                    'periccrh': ('Pericalcarine Cortex', 'L'), 
                    'pllh': ('Pallidum', 'L'),
                    'plrh': ('Pallidum', 'L'), 
                    'postcentrallh': ('Postcentral Gyrus', 'L'), 
                    'postcentralrh': ('Postcentral Gyrus', 'R'), 
                    'postcnlh': ('Postcentral Gyrus', 'L'), 
                    'postcnrh': ('Postcentral Gyrus', 'R'),
                    'precentrallh': ('Precentral Gyrus', 'L'), 
                    'precentralrh': ('Precentral Gyrus', 'R'), 
                    'precnlh': ('Precentral Gyrus', 'L'), 
                    'precnrh': ('Precentral Gyrus', 'R'),
                    'precuneuslh': ('Precuneus', 'L'), 
                    'precuneusrh': ('Precuneus', 'L'), 
                    'psclatelh': ('Cingulate Gyrus, Posterior', 'L'), 
                    'psclaterh': ('Cingulate Gyrus, Posterior', 'R'), 
                    'pscslh': ('Superior Corticostriate Tract, Parietal', 'L'),
                    'pscsrh': ('Superior Corticostriate Tract, Parietal', 'R'), 
                    'pslflh': ('Superior Longitudinal Fasiculus, Parietal', 'L'), 
                    'pslfrh': ('Superior Longitudinal Fasiculus, Parietal', 'R'), 
                    'ptcatelh': ('Cingulate Gyrus, Posterior', 'L'), 
                    'ptcaterh': ('Cingulate Gyrus, Posterior', 'R'), 
                    'ptlh': ('Putamen', 'L'),
                    'ptoltmlh': (), 
                    'ptoltmrh': (), 
                    'ptrh': ('Putamen', 'R'), 
                    'putamenlh': ('Putamen', 'L'), 
                    'putamenrh': ('Putamen', 'R'),
                    'rlaclatelh': ('Cingulate Gyrus, Rostral Anterior', 'L'), 
                    'rlaclaterh': ('Cingulate Gyrus, Rostral Anterior', 'R'), 
                    'rlmdflh': ('Middle Frontal Gyrus, Rostral', 'L'), 
                    'rlmdfrh': ('Middle Frontal Gyrus, Rostral', 'R'), 
                    'rracatelh': ('Cingulate Gyrus, Rostral Anterior', 'L'),
                    'rracaterh': ('Cingulate Gyrus, Rostral Anterior', 'R'), 
                    'rrmdfrlh': ('Middle Frontal Gyrus, Rostral', 'L'), 
                    'rrmdfrrh': ('Middle Frontal Gyrus, Rostral', 'R'), 
                    'rspltp': ('Retrosplenial Temporal Network', 'B'), 
                    'sa': ('Salience Network', 'B'), 
                    'scslh': ('Superior Corticostriate Tract', 'L'),
                    'scsrh': ('Superior Corticostriate Tract', 'L'), 
                    'sifclh': ('Striatum, Inferior Frontal', 'L'), 
                    'sifcrh': ('Striatum, Inferior Frontal', 'L'), 
                    'slflh': ('Superior Longitudinal Fasiculus', 'L'), 
                    'slfrh': ('Superior Longitudinal Fasiculus', 'R'), 
                    'smh': ('Sensorimotor Network, Hand', 'B'), 
                    'smlh': ('Supramarginal Gyrus', 'L'),
                    'smm': ('Sensorimotor Network, Mouth', 'B'), 
                    'smrh': ('Supramarginal Gyrus', 'R'), 
                    'spetallh': ('Superior Parietal Lobule', 'L'), 
                    'spetalrh': ('Superior Parietal Lobule', 'R'), 
                    'suflh': ('Superior Frontal Gyrus', 'L'), 
                    'sufrh': ('Superior Frontal Gyrus', 'R'), 
                    'sufrlh': ('Superior Frontal Gyrus', 'L'),
                    'sufrrh': ('Superior Frontal Gyrus', 'R'), 
                    'supllh': ('Superior Parietal Lobule', 'L'), 
                    'suplrh': ('Superior Parietal Lobule', 'R'), 
                    'sutlh': ('Superior Temporal Gyrus', 'L'), 
                    'sutmlh': ('Superior Temporal Gyrus', 'L'), 
                    'sutmrh': ('Superior Temporal Gyrus', 'R'), 
                    'sutrh': ('Superior Temporal Gyrus', 'R'),
                    'thplh': ('Thalamus', 'L'), 
                    'thprh': ('Thalamus', 'R'), 
                    'tmpolelh': ('Temporal Pole', 'L'), 
                    'tmpolerh': ('Temporal Pole', 'R'), 
                    'total': (), 
                    'tplh': ('Thalamus', 'L'),
                    'tpolelh': ('Temporal Pole', 'L'), 
                    'tpolerh': ('Temporal Pole', 'R'), 
                    'tprh': ('Thalamus', 'R'), 
                    'trvtmlh': ('Transverse Temporal Gyrus', 'L'), 
                    'trvtmrh': ('Transverse Temporal Gyrus', 'R'), 
                    'tslflh': ('Superior Longitudinal Fasiculus, Temporal', 'L'),
                    'tslfrh': ('Superior Longitudinal Fasiculus, Temporal', 'R'), 
                    'tvtlh': ('Transverse Temporal Gyrus', 'L'), 
                    'tvtrh': ('Transverse Temporal Gyrus', 'R'), 
                    'unclh': ('Uncinate Fasiculus', 'L'), 
                    'uncrh': ('Uncinate Fasiculus', 'R'), 
                    'vdclh': ('Ventral Diencephalon', 'L'), 
                    'vdcrh': ('Ventral Diencephalon', 'R'),
                    'vedclh': ('Ventral Diencephalon', 'L'), 
                    'vedcrh': ('Ventral Diencephalon', 'R'), 
                    'ventraldclh': ('Ventral Diencephalon', 'L'), 
                    'ventraldcrh': ('Ventral Diencephalon', 'R'), 
                    'vs': ('Visual Network', 'B'), 
                    'vta': ('Ventral Attention Network', 'B'),
                    'vtdclh': ('Ventral Diencephalon', 'L'), 
                    'vtdcrh': ('Ventral Diencephalon', 'R'),
                    'ad_ngd_ad': ('Auditory Network, Within', 'B'),
                    'ca': ('Cinguloparietal Network', 'B'),
                    'cerc': ('Cingulo-opercular Network', 'B'),
                    'cgc': ('Cingulo-opercular Network', 'B'),
                    'cmfrh': ('Middle Frontal Gyrus, Caudal', 'R'),
                    'cnlh': ('Cuneus', 'L'),
                    'cnrh': ('Cuneus', 'R'),
                    'copa': ('Cinguloparietal Network', 'B'),
                    'pcclh': ('Pericalcarine', 'L'),
                    'pccrh': ('Pericalcarine', 'R'),
                    'pcglh': ('Cingulate Gyrus, Posterior', 'L'),
                    'pcgrh': ('Cingulate Gyrus, Posterior', 'R'),
                    'pctlh': ('Postcentral Gyrus', 'L'),
                    'pctrh': ('Postcentral Gyrus', 'R'),
                    'phlh': ('Parahippocampal Gyrus', 'L'),
                    'phrh': ('Parahippocampal Gyrus', 'R'),
                    'poblh': ('Inferior Frontal Gyrus, Pars Orbitalis', 'L'),
                    'pobrh': ('Inferior Frontal Gyrus, Pars Orbitalis', 'R'),
                    'poplh': ('Inferior Frontal Gyrus, Pars Opercularis', 'L'),
                    'poprh': ('Inferior Frontal Gyrus, Pars Opercularis', 'R'),
                    'prcnlh': ('Precuneus', 'L'),
                    'prcnrh': ('Precuneus', 'R'),
                    'prctlh': ('Precentral Gyrus', 'L'),
                    'prctrh': ('Precentral Gyrus', 'R'),
                    'ptglh': ('Inferior Frontal Gyrus, Pars Triangularis', 'L'),
                    'ptgrh': ('Inferior Frontal Gyrus, Pars Triangularis', 'R'),
                    'raclh': ('Cingulate Gyrus, Rostral Anterior', 'L'),
                    'racrh': ('Cingulate Gyrus, Rostral Anterior', 'R'),
                    'rmflh': ('Middle Frontal Gyrus, Rostral', 'L'),
                    'rmfrh': ('Middle Frontal Gyrus, Rostral', 'R'),
                    'sflh': ('Superior Frontal Gyrus', 'L'),
                    'sfrh': ('Superior Frontal Gyrus', 'R'),
                    'splh': ('Superior Parietal Lobule', 'L'),
                    'sprh': ('Superior Parietal Lobule', 'R'),
                    'stlh': ('Superior Temporal Lobule', 'L'),
                    'strh': ('Superior Temporal Lobule', 'R'),
                    'ttlh': ('Transverse Temporal Lobe', 'L'),
                    'ttrh': ('Transverse Temporal Lobe', 'R'),
                    'erlh': ('Entorhinal Cortex', 'L'),
                    'errh': ('Entorhinal Cortex', 'R'),'fplh': ('Frontal Pole', 'L'),
                    'fprh': ('Frontal Pole', 'R'),
                    'au': ('Auditory Network', 'B'),
                    'dsa': ('Dorsal Attention Network', 'B'),
                    'fopa': ('Frontoparietal Network', 'B'),
                    'none': ('Extra-Network', 'B'),
                    'rst': ('Retrosplenial Temporal Network', 'B'),
                    'iclh': ('Cingulate Gyrus, Ithmus', 'L'),
                    'icrh': ('Cingulate Gyrus, Ithmus', 'R'),
                    'iplh': ('Inferior Parietal Lobule', 'L'),
                    'iprh': ('Inferior Parietal Lobule', 'R'),
                    'islh': ('Insula', 'L'),
                    'isrh': ('Insula', 'R'),
                    'itlh': ('Inferior Temporal Gyrus', 'L'),
                    'itrh': ('Inferior Temporal Gyrus', 'R'),
                    'lglh': ('Lingual Gyrus', 'L'),
                    'lgrh': ('Lingual Gyrus', 'R'),
                    'loflh': ('Orbitofrontal Gyrus, Lateral', 'L'),
                    'lofrh': ('Orbitofrontal Gyrus, Lateral', 'R'),
                    'lolh': ('Lateral Occipital Gyrus', 'L'),
                    'lorh': ('Lateral Occipital Gyrus', 'R'),
                    'moflh': ('Orbitofrontal Gyrus, Medial', 'L'),
                    'mofrh': ('Orbitofrontal Gyrus, Medial', 'R'),
                    'mtlh': ('Middle Temporal Gyrus', 'L'),
                    'mtrh': ('Middle Temporal Gyrus', 'R'),}
    missing = []
    for i in df.index: 
        if '_scs_' in df.loc[i]['region']:
            temp = df.loc[i]['region'].split('_scs_')
            region_name = f'{region_names[temp[0]][0]}, {region_names[temp[1]][0]}'
            hemisphere = region_names[temp[1]][1]
            df.at[i, 'long_region'] = region_name
            df.at[i, 'hemisphere'] = hemisphere
        elif '_ngd_' in df.loc[i]['region']:
            temp = df.loc[i]['region'].split('_ngd_')
            region_name = f'{region_names[temp[0]][0]}, {region_names[temp[1]][0]}'
            hemisphere = region_names[temp[1]][1]
            df.at[i, 'long_region'] = region_name
            df.at[i, 'hemisphere'] = hemisphere
        elif df.loc[i]['region'] not in region_names.keys():
            missing.append(df.loc[i]['region'])
        else:
            long_region = region_names[df.loc[i]['region']]
            df.at[i, 'long_region'] = long_region[0]
            df.at[i, 'hemisphere'] = long_region[1]
    if missing:
        return df, missing
    else:
        return df

In [63]:
stat_df

,mean,sdev,max,min,25%,75%,modality,measure,atlas,region,concept,long_region,hemisphere
smri_area_cdk_banksstslh.change_score,-0.329258,2.782295,-20.134047,19.887159,-1.638505,1.056500,smri,area,cdk,banksstslh,macrostructure,Banks of Superior Temporal Sulcus,L
smri_area_cdk_cdacatelh.change_score,0.724602,2.611420,-28.767677,21.583172,-0.735880,2.191229,smri,area,cdk,cdacatelh,macrostructure,"Anterior Cingulate, Caudal",L
smri_area_cdk_cdmdfrlh.change_score,0.185836,2.766865,-29.224690,26.461783,-0.684505,1.132905,smri,area,cdk,cdmdfrlh,macrostructure,"Middle Frontal Gyrus, Caudal",L
smri_area_cdk_cuneuslh.change_score,0.016584,1.771285,-20.483797,29.078628,-0.895536,1.007698,smri,area,cdk,cuneuslh,macrostructure,Cuneus,L
smri_area_cdk_ehinallh.change_score,0.546391,5.415478,-34.011429,42.996639,-2.494257,3.556375,smri,area,cdk,ehinallh,macrostructure,Entorhinal Cortex,L
...,...,...,...,...,...,...,...,...,...,...,...,...,...
rsfmri_cor_ngd_vs_scs_plrh.change_score,3.476936,51.177926,-127.999852,129.956447,-35.994852,43.787573,rsfmri,cor,ngd,vs_scs_plrh,function,"Visual Network, Pallidum",L
rsfmri_cor_ngd_vs_scs_hprh.change_score,-1.094951,51.105059,-114.108845,114.360318,-40.521594,38.775876,rsfmri,cor,ngd,vs_scs_hprh,function,"Visual Network, Hippocampus",R
rsfmri_cor_ngd_vs_scs_agrh.change_score,-0.265575,47.308354,-118.680739,113.762235,-34.183868,33.585765,rsfmri,cor,ngd,vs_scs_agrh,function,"Visual Network, Amygdala",R
rsfmri_cor_ngd_vs_scs_aarh.change_score,-1.870387,51.699947,-113.539713,129.075027,-42.098646,38.042599,rsfmri,cor,ngd,vs_scs_aarh,function,"Visual Network, Accumbens",R


In [64]:
stat_df.to_csv(join(PROJ_DIR, OUTP_DIR, 'change_score_descriptives_region_names.csv'))